In [1]:
#importing libraries
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from keras.backend import dropout
import keras
import random
from sklearn.model_selection import train_test_split

2024-03-03 11:30:01.854336: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 11:30:02.457340: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-03-03 11:30:02.457386: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-03-03 11:30:02.457391: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# Set a seed value
seed_value= 2

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

# 5. For layers that introduce randomness like dropout, make sure to set seed values 
# model.add(Dropout(0.25, seed=seed_value))

In [3]:
#import datasets

train = pd.read_csv("trainsplitfinal.csv")
test = pd.read_csv("testsplitfinal.csv")
val = pd.read_csv("valsplitfinal.csv")

In [4]:
#shuffle the datasets before splitting into X and Y
train = train.sample(frac=1, random_state=42)
test = test.sample(frac=1, random_state=42)
val = val.sample(frac=1, random_state=42)

In [5]:
train.groupby(["labels"]).size()

labels
L_Alanine          92
L_Arginine         99
L_Asparagine       12
L_Cystine           9
L_Glutamate        18
L_Glutamine        15
L_Histidine        15
L_Isoleucine       26
L_Leucine          26
L_Lysine           29
L_Methionine       12
L_Phenylalanine    26
L_Proline          15
L_Serine           12
L_Threonine        12
L_Tryptophan        9
L_Valine           12
dtype: int64

In [6]:
test.groupby(["labels"]).size()

labels
L_Alanine          20
L_Arginine         21
L_Asparagine        3
L_Cystine           2
L_Glutamate         4
L_Glutamine         3
L_Histidine         3
L_Isoleucine        5
L_Leucine           5
L_Lysine            6
L_Methionine        3
L_Phenylalanine     5
L_Proline           3
L_Serine            3
L_Threonine         3
L_Tryptophan        2
L_Valine            3
dtype: int64

In [7]:
val.groupby(["labels"]).size()

labels
L_Alanine          20
L_Arginine         22
L_Asparagine        2
L_Cystine           1
L_Glutamate         4
L_Glutamine         4
L_Histidine         4
L_Isoleucine        6
L_Leucine           6
L_Lysine            7
L_Methionine        2
L_Phenylalanine     6
L_Proline           4
L_Serine            2
L_Threonine         2
L_Tryptophan        1
L_Valine            2
dtype: int64

In [8]:
#train
#one hot encode labels
df_dummies_train = pd.get_dummies(train['labels'])
df_class_neural_train = pd.concat([train,df_dummies_train],axis=1)
df_class_neural_train.head()

,0,1,2,3,4,5,6,7,8,9,...,L_Isoleucine,L_Leucine,L_Lysine,L_Methionine,L_Phenylalanine,L_Proline,L_Serine,L_Threonine,L_Tryptophan,L_Valine
265,-0.015371,-0.050200,-0.689597,-0.645556,-1.030798,-0.624544,-0.557602,-0.310951,-0.244630,0.326894,...,0,0,0,0,0,0,0,0,0,0
78,0.533713,0.369027,1.096170,1.192440,-0.447995,-0.370572,-1.218716,-0.394225,1.039009,1.335908,...,0,0,0,0,0,0,0,0,0,0
347,0.915757,0.499415,0.300768,-0.248091,-0.539519,-0.372294,-0.289839,0.142104,-0.130767,1.012644,...,0,0,0,0,0,0,0,0,0,0
255,0.417486,0.330047,0.320320,0.491617,0.259264,0.472199,-0.165878,0.793265,0.045475,0.414088,...,0,0,1,0,0,0,0,0,0,0
327,-0.245026,-1.013841,-0.429411,-0.587188,-0.198697,-0.281615,0.494006,0.294681,1.453202,1.439727,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#test
#one hot encode labels
df_dummies_test = pd.get_dummies(test['labels'])
df_class_neural_test = pd.concat([test,df_dummies_test],axis=1)
df_class_neural_test.head()

,0,1,2,3,4,5,6,7,8,9,...,L_Isoleucine,L_Leucine,L_Lysine,L_Methionine,L_Phenylalanine,L_Proline,L_Serine,L_Threonine,L_Tryptophan,L_Valine
40,-0.483016,-0.537050,-1.025172,-1.124405,-1.157614,-1.169983,-1.071377,-0.981773,-0.630656,-0.769079,...,0,0,0,0,0,0,0,0,0,0
22,0.126533,-0.765194,-0.737649,-0.795911,-0.186244,-0.121682,0.286255,0.598491,0.983311,0.840110,...,0,0,0,0,0,0,0,0,0,0
55,0.567126,0.804352,-1.151460,-0.765799,-0.837458,-0.284110,-0.813847,0.937344,1.304037,1.755969,...,0,0,0,0,0,0,0,0,0,0
72,0.170564,-0.420655,0.262387,-0.646842,-0.639161,-1.545182,-0.359907,-0.964754,0.078531,0.254270,...,0,0,0,0,0,1,0,0,0,0
0,0.487114,0.326123,0.368290,0.345744,0.681846,0.252896,0.288003,0.896431,1.243423,1.103331,...,0,0,0,0,0,0,0,0,0,0


In [10]:
#val
#one hot encode labels
df_dummies_val = pd.get_dummies(val['labels'])
df_class_neural_val = pd.concat([val,df_dummies_val],axis=1)
df_class_neural_val.head()

,0,1,2,3,4,5,6,7,8,9,...,L_Isoleucine,L_Leucine,L_Lysine,L_Methionine,L_Phenylalanine,L_Proline,L_Serine,L_Threonine,L_Tryptophan,L_Valine
68,0.526471,-0.442709,0.315680,-0.285354,-0.583893,-0.569704,0.603291,0.497402,0.719592,1.613877,...,0,1,0,0,0,0,0,0,0,0
22,1.249557,1.143450,0.936365,0.185062,0.280913,0.357344,0.488149,-0.095665,-0.361493,-0.016594,...,0,0,0,0,0,0,0,0,1,0
72,0.872252,0.436853,0.188857,-0.209450,-0.178809,-0.221154,-0.427222,0.416149,0.348627,0.650577,...,0,0,0,0,0,0,0,0,0,0
73,0.481580,0.042152,0.168081,-0.725333,-0.319640,-0.419307,0.270552,-0.610970,-0.018212,-0.303933,...,0,0,1,0,0,0,0,0,0,0
0,0.530004,0.642162,0.372406,0.045927,-0.781801,-0.362716,0.010942,0.698137,0.846708,0.851190,...,0,0,0,0,0,0,0,0,0,0


In [11]:
X_train = df_class_neural_train.iloc[:,:72]
y_train = df_class_neural_train.iloc[:,73:]
X_test = df_class_neural_test.iloc[:,:72]
y_test = df_class_neural_test.iloc[:,73:]
X_val = df_class_neural_val.iloc[:,:72]
y_val = df_class_neural_val.iloc[:,73:]

In [12]:
#check shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(439, 72)
(439, 17)
(94, 72)
(94, 17)
(95, 72)
(95, 17)


In [13]:
#save the splits
X_train.to_csv("finalXtrain.csv",index=False)
y_train.to_csv("finalYtrain.csv",index=False)
X_test.to_csv("finalXtest.csv",index=False)
y_test.to_csv("finalYtest.csv",index=False)
X_val.to_csv("finalXval.csv",index=False)
y_val.to_csv("finalYval.csv",index=False)